# JDC행사정보 크롤링 모듈<br>
- https://www.jdcenter.com/prcenter/issue/notice.cs 를 대상합니다.<br><br>
- 제목, 주제, 시간, 장소, 대상자, 방법에 해당하는 텍스트만 파싱하여 DB에 채워넣는 것을 목표로 합니다. <br><br>
- 동적으로 웹을 그리는 까닭에 selenium 모듈을 사용하였습니다.<br><br>
- 크롤링한 텍스트를 파싱하기 위해 beautifulsoup4 라이브러리를 사용하였습니다.<br><br>
- 해당 프로그램에서 driver.implicitly_wait()이 작동하지 않아서 time.sleep()을 사용하였습니다.

In [1]:
from selenium import webdriver
import bs4
from time import sleep

In [2]:
#게시글 제목과 전문을 파싱하여 채워넣기 위한 배열입니다.
title = []
body = []
url = []

In [3]:
#게시글을 해당 속성을 파싱하여 채워넣기 위한 배열입니다.
time_set = []
howto_set = []
place_set = []
targeter_set = []
topic_set = []

## Before executing Selenium <br>
- selenium 라이브러리는 파싱을 목적으로하는 라이브러리가 아닌 웹을 제어하는 라이브러리 입니다.<br><br>
- selenium 라이브러리는 전적으로 웹 브라우저에 의존하여 동작합니다.<br><br>
- 웹 드라이버를 첨부하였습니다. 필자의 경우 크롬 79.0 버전을 사용하였습니다. 자신이 사용하는 브라우저와 버전에 맞추어 저장하도록 합니다.<br>

In [4]:
#글의 제목과 게시글 전문을 크롤링하는 코드입니다.
driver = webdriver.Chrome('C:\웹 드라이버\chromedriver.exe')#웹 드라이버가 저장되는 경로입니다.
driver.implicitly_wait(1)
driver.get('https://www.jdcenter.com/prcenter/issue/notice.cs')
    
html = driver.page_source
soup = bs4.BeautifulSoup(html, 'html.parser')

sleep(5)    #IDLE TIME은 자신의 인터넷 속도에 맞추어 유동적으로 할당해도 좋습니다.

# 해당 반복문은 게시글 전체(266개)를 모두 탐색할 수 있습니다. 지금은 간단히 9개의 게시글을 탐색하도록 합니다.
for i in range(1, 10): 
    
    # 2 ~ 25 페이지를 클릭해주는 코드 입니다.
    if 20 >= i > 10:
        next_page = driver.find_element_by_xpath('//*[@id="pagination"]/div/div/a[{}]'.format(2))
        next_page.click() 
        sleep(3)
    if 30 >= i > 20:
        next_page = driver.find_element_by_xpath('//*[@id="pagination"]/div/div/a[{}]'.format(3))
        next_page.click() 
        sleep(3)
    if 40 >= i > 30:
        next_page = driver.find_element_by_xpath('//*[@id="pagination"]/div/div/a[{}]'.format(4))
        next_page.click() 
        sleep(3)
    if i > 40:
        next_page = driver.find_element_by_xpath('//*[@id="pagination"]/div/div/a[{}]'.format(5))
        next_page.click() 
        sleep(3)
    if i > 50:
        next_page = driver.find_element_by_xpath('//*[@id="pagination"]/div/a[2]')
        next_page.click()
        sleep(3)
    if i > 60:
        if i % 10 == 0:
            for k in range(0, (i//10)-6):
                next_page = driver.find_element_by_xpath('//*[@id="pagination"]/div/a[3]')
                next_page.click()
                sleep(4)
        else:
            for k in range(0, (i//10)-5):
                next_page = driver.find_element_by_xpath('//*[@id="pagination"]/div/a[3]')
                next_page.click()
                sleep(4)         
    #------------------
    
    j = i%10
    if j == 0:
        j = 10

    sleep(2)
    post_page = driver.find_element_by_xpath('//*[@id="datatable"]/tbody/tr[{}]/td[2]/a'.format(j))
    post_page.click()

    sleep(2)
    driver.get(driver.current_url)
    html = driver.page_source
    soup = bs4.BeautifulSoup(html, 'html.parser')

    sleep(2)
    h3 = soup.find("h3")
    title.append(h3.text)

    bod = soup.find('div', class_='bod_bd')
    body.append(bod.text)
    
    url.append(driver.current_url)

    driver.back()
    sleep(2)
    
    print('[+]게시글 '+ str(i) +'번째를 파싱했습니다[+]')
#title과 body 배열에 각각의 요소를 파싱했습니다.

[+]게시글 1번째를 파싱했습니다[+]
[+]게시글 2번째를 파싱했습니다[+]
[+]게시글 3번째를 파싱했습니다[+]
[+]게시글 4번째를 파싱했습니다[+]
[+]게시글 5번째를 파싱했습니다[+]
[+]게시글 6번째를 파싱했습니다[+]
[+]게시글 7번째를 파싱했습니다[+]
[+]게시글 8번째를 파싱했습니다[+]
[+]게시글 9번째를 파싱했습니다[+]


In [12]:
#파싱된 텍스트중 \xa0을 지우는 코드 입니다.
for i in range(0, len(body)):
    body[i] = body[i].replace("\xa0","")

In [6]:
#파싱된 텍스트중 \n\n을 줄이는 코드 입니다.
for i in range(0, len(body)):
    body[i] = body[i].replace("\n\n","\n")

## 해당 텍스트 파싱 알고리즘<BR>
- 정규식을 이용하여 파싱하였습니다.<BR><br>
- 게시글 전문 중 해당 텍스트를 포함하고 있는 게시글을 한 줄 때어내어 파싱합니다.<BR><br>
- 예를들어, 신청방법의 경우 '접수 방법', '참가 신청', '신청 방법' 등의 텍스트를 포함합니다. 게시글 중 해당 텍스트를 포함하는 텍스트를 한줄 때어다가 배열에 채워넣는 방식입니다.

In [7]:
#날짜부분 파싱 알고리즘
import re

time = re.compile('\d\w*\s*\d')
date_catch = re.compile('기\s*간|일\s*시|일자\s+|접수일|접수기간|접수마감|공고일|공고기간|기한|모집기간|일정')

i = 0
for test in body:
    text = test.split('\n')
    print(i)
    i += 1
    temp = []
    for t in text:
        if(time.search(t) and date_catch.search(t)):
            print(str(t))
            temp.append(t)
            
    time_set.append(temp)
    
    
#--------------------

0
ㅇ 행복주택 입주자 모집 신청 일시 : 2020.2.14.(금)~2.17(월), 10:00~17:00(주말포함)
ㅇ 10년공공임대 계약 체결 일시 : 2020.2.17.(월)~2.20(목), 10:00~16:00
2. 또한 접수인의 안전을 위해 접수기간 동안 방문 시 필히 마스크 착용을 부탁드리며,최근 14일 이내 중국 등을 방문하거나 발열(37.5℃ 이상)또는 호흡기증상이 있으신 경우 현장 접수를 제한하고 있습니다.
1
2. 사업기간 : 2020. 02. ~ 2020. 10.
ㅇ 접수기간 : 2020. 2. 11.(화)~ 2. 21.(금)
2
2. 사업기간 : 2020. 02. ~ 2020. 10.
3
가. 모집공고일: 2020.02.04.(화) ~ 2020.02.13.(목)
마. 신청 접수일: 2020.02.14.(금) ~ 2020.02.17.(월), 10:00~17:00
4
			□ JDC의 행복주택 공고일이 당초 일정보다 일주일 뒤인 2020년 2월 4일로 변경됨을 알려드립니다.
5
6
7
ㅇ 접수기간 : 2020.01.15.(수) ~ 2020.02.14.(금) 낮 12시
ㅇ 일시 : 2020.02.04. (화) 1차 10시 /2차 16시
8


In [8]:
#신청방법 부분 파싱 알고리즘
howto_catch = re.compile('접수\s*방법|참가\s*신청|방문|신청\s*방법|접수안내|응모\s*방법|모집\s*방법|지원\s*방법')

i = 0
for test in body:
    text = test.split('\n')
    print(i)
    i += 1
    temp = []
    for t in text:
        if(howto_catch.search(t)):
            print(t)
            temp.append(t)
            if(len(t)<=10):
                print(text[text.index(t)+1])
                temp.append(text[text.index(t)+1])
            
    howto_set.append(temp)
#-------------------

0
2. 또한 접수인의 안전을 위해 접수기간 동안 방문 시 필히 마스크 착용을 부탁드리며,최근 14일 이내 중국 등을 방문하거나 발열(37.5℃ 이상)또는 호흡기증상이 있으신 경우 현장 접수를 제한하고 있습니다.
※ 방문시 엘리트 빌딩 1층(로비) 정문을 통해서 만 출입이 가능합니다.
1
ㅇ 접수방법 : 방문, 우편, 이메일 접수 가능
- 우편 & 방문접수: 제주시 도남로 121. 3층
2
3
다. 모집방법: 현장 접수(JDC 엘리트 빌딩 4층 대강당) 또는 우편접수
4
5
6
7
2. 접수방법
ㅇ 접수기간 : 2020.01.15.(수) ~ 2020.02.14.(금) 낮 12시
ㅇ 접수방법 : 구글 설문지 제출(수시접수)
8


In [9]:
#장소 부분 파싱 알고리즘
place_catch = re.compile('장\s*소|축제\s*장소')

i = 0
for test in body:
    text = test.split('\n')
    print(i)
    i += 1
    temp = []
    for t in text:
        if(place_catch.search(t)):
            print(t)
            temp.append(t)
            if(len(t)<=10):
                print(text[text.index(t)+1])
                temp.append(text[text.index(t)+1])
            
    place_set.append(temp)
#-------------------

0
JDC 공공임대주택 관련 현장접수 및 계약장소 변경 공지
1
2
3
4
5
6
7
ㅇ 장소 : (1차) 제주 사회적경제지원센터 내 몬딱가공소 대강당 (제주시 중앙로 165번지 고용복지플러스센터 1층)
8


In [10]:
#대상자 부분 파싱 알고리즘
targeter_catch = re.compile('참가\s*자격|지원\s*자격|지원\s*대상|신청\s*자격|지원\s*대상|참여\w*대상|참석\w*대상|모집\s*대상|자격\s*요건|공모\s*대상')

i = 0
for test in body:
    text = test.split('\n')
    print(i)
    i += 1
    temp = []
    for t in text:
        if(targeter_catch.search(t)):
            print(t)
            temp.append(t)
            if(len(t)<=10):
                print(text[text.index(t)+1])
                temp.append(text[text.index(t)+1])
                
    targeter_set.append(temp)
#-------------------

0
1
3. 지원대상 : 도내 취약계층 아동가구
1. 지원대상 신청조건
2
3. 지원대상 : 도내 취약계층 아동가구
3
라. 모집대상: 입주자격을 갖춘 자(모집 공고문 참조)
4
5
6
7
			 JDC 제3차 사회적경제조직 지원사업 지원대상 모집 공고
제주국제자유도시개발센터(JDC)가 지원하고 (재)한국사회투자가 수탁 운영하는 「JDC 제2차 사회적경제조직 지원사업」의 지원대상을 다음과 같이 공고합니다.
ㅇ (지원대상) 제주 소재 성장기 사회적경제조직
- 지원규모 : 지원대상별 최소 30백만원 ~ 최대 100백만원
8


In [11]:
#주제 부분 파싱 알고리즘
topic_catch = re.compile('사업\s*내용|응모\s*주제|사업\s*목적\s+|연수\s*내용|추진\s*내용|지원\s*내용|시상\s*내용|공사\s*내용|행사\s*내용|대회\s*주제|토론\s*주제|스피치\s*주제|응모\s*주제|강의\s*주제|대\s*주제|소\s*주제|공모\s*주제|주요\s*활동|활동\s*내용|활동\s*개요|\s+주\s*제\s+')

i = 0
for test in body:
    text = test.split('\n')
    print(i)
    i += 1
    temp = []
    for t in text:
        if(topic_catch.search(t)):
            print(t)
            temp.append(t)
            if(len(t)<=10):
                print(text[text.index(t)+1])
                temp.append(text[text.index(t)+1])
                
    topic_set.append(temp)
#-------------------

0
1
□ 주요 사업 내용
1. 지원규모: 아동가정 가구 21세대 (세대당 최대 600만원)
2. 지원내용 : 주거시설 성능개선 및 생활편의 개선 등에 지원
2
3
4
5
6
7
ㅇ신청서의 모든 내용은 객관적으로 입증할 수 있어야 하며, 사업 내용 및 제출서류가 허위로 밝혀질 경우 (재)한국사회투자는
8


In [13]:
#test it out
for i in range(0, len(body)):
    print(' '+str(i)+'번째 게시글 입니다 [+] \n')
    print(title[i].strip())
    print("\n".join([str(i) for i in topic_set[i]] ))
    print("\n".join([str(i) for i in time_set[i]] ))
    print("\n".join([str(i) for i in howto_set[i]] ))
    print("\n".join([str(i) for i in place_set[i]] ))
    print("\n".join([str(i) for i in targeter_set[i]] ))
    print('-------------------\n')
    
#관리자 페이지에서 보여질 게시글은 아래와 같습니다.

 0번째 게시글 입니다 [+] 

JDC 공공임대주택 관련 현장접수 및 계약장소 변경 공지

ㅇ 행복주택 입주자 모집 신청 일시 : 2020.2.14.(금)~2.17(월), 10:00~17:00(주말포함)
ㅇ 10년공공임대 계약 체결 일시 : 2020.2.17.(월)~2.20(목), 10:00~16:00
2. 또한 접수인의 안전을 위해 접수기간 동안 방문 시 필히 마스크 착용을 부탁드리며,최근 14일 이내 중국 등을 방문하거나 발열(37.5℃ 이상)또는 호흡기증상이 있으신 경우 현장 접수를 제한하고 있습니다.
2. 또한 접수인의 안전을 위해 접수기간 동안 방문 시 필히 마스크 착용을 부탁드리며,최근 14일 이내 중국 등을 방문하거나 발열(37.5℃ 이상)또는 호흡기증상이 있으신 경우 현장 접수를 제한하고 있습니다.
※ 방문시 엘리트 빌딩 1층(로비) 정문을 통해서 만 출입이 가능합니다.
JDC 공공임대주택 관련 현장접수 및 계약장소 변경 공지

-------------------

 1번째 게시글 입니다 [+] 

JDC 사랑의리모델링 지원 신청자 모집
□ 주요 사업 내용
1. 지원규모: 아동가정 가구 21세대 (세대당 최대 600만원)
2. 지원내용 : 주거시설 성능개선 및 생활편의 개선 등에 지원
2. 사업기간 : 2020. 02. ~ 2020. 10.
ㅇ 접수기간 : 2020. 2. 11.(화)~ 2. 21.(금)
ㅇ 접수방법 : 방문, 우편, 이메일 접수 가능
- 우편 & 방문접수: 제주시 도남로 121. 3층

3. 지원대상 : 도내 취약계층 아동가구
1. 지원대상 신청조건
-------------------

 2번째 게시글 입니다 [+] 

JDC '사랑의 리모델링' 지원 신청자 모집

2. 사업기간 : 2020. 02. ~ 2020. 10.


3. 지원대상 : 도내 취약계층 아동가구
-------------------

 3번째 게시글 입니다 [+] 

제주첨단과학기술단지 A23블록 「JDC 제주첨단 행복주택」입주자 모집 공고

가. 모집공고일

## 서버API 호출 <br>
- 해당 서비스 DB에 채워넣기 위해 API서버를 호출하는 코드입니다.<br><br>
- 해당 텍스트는 관리자 페이지에서 검수를 받아 어플리케이션에서 최종적으로 보여집니다.<br><br>
- API를 호출하기위해 requests 라이브러리를 사용하였습니다.

In [14]:
import requests

#파싱한 텍스트를 서버API를 호출하여 DB에 밀어 넣습니다.
for i in range(0, 1):
    data = {
        "post_url":url[i],
        "replace":"ture",
        "candidate": [
            {
                "type":"title",
                "value":"{}번째 테스트 게시글입니다.".format(str(i+1))
            },
            {
                "type":"title",
                "value":title[i].strip()
            },
            {
                "type":"date",
                "value":time_set[i]
            },
            {
                "type":"location",
                "value":place_set[i]+howto_set[i]
            },
            {
                "type":"target",
                "value":targeter_set[i]
            },
            {
                "type":"content",
                "value":topic_set[i]
            }
        ],
        "original_text":body[i]
    }

    r = requests.post('http://13.124.116.158:3000/create_campaign', json = data)

    url = 'http://13.124.116.158:3000/create_campaign'

## 크롤링 & 파싱 통계 <br>

전체 게시글 266개<br>
추출에 성공한 게시글 187개<br>
하나도 추출하지 못한 게시글 79개 <br>
<br>
각 항목에서 <br>
topic 실패 200개<br>
date 실패 89개<br>
howto 실패 152개<br>
place 실패 183개<br>
targeter 실패 182개<br>
<br>
추출 불가능 게시글(첨부파일, 이미지) 32개<br>
결과발표 혹은 감사인사,기념사,취소안내,위험안내 등 47개<br>
<br>
전체 게시글 중 파싱에 성공한 경우 70.3% (187/266)